In [ ]:
# CÉLULA 1 - MONTAGEM DO GOOGLE DRIVE E DEFINIÇÃO DE CAMINHOS GLOBAIS
from google.colab import drive #importa biblioteca para conectar Colab com Drive (onde o database está)
drive.mount('/content/drive') #acessa meu Drive como uma pasta local

# Defina os caminhos para suas pastas e arquivos no Google Drive
  #Armazena na variável o caminho do database
DATA_PATH = "/content/drive/MyDrive/InfoBio/TCC 2/databaseRag/artigos"
  #Armazena na variável o caminho do model LLM (que está no drive)
LLAMA_MODEL_PATH = "/content/drive/MyDrive/InfoBio/TCC 2/databaseRag/models/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf"
  #Armazena na variável o caminho onde o banco ChromaDB será salvo. O Chroma vai guardar os vetores numéricos que eram os arigos
CHROMA_PERSIST_DIR = "/content/drive/MyDrive/InfoBio/TCC 2/databaseRag/chroma_db"

#Printar na tela os valores das variáveis (útil para verificação, se foi linkado com sucesso)
print(f"Caminho dos Dados: {DATA_PATH}")
print(f"Caminho do Modelo LLM: {LLAMA_MODEL_PATH}")
print(f"Caminho do Banco de Dados Chroma: {CHROMA_PERSIST_DIR}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Caminho dos Dados: /content/drive/MyDrive/InfoBio/TCC 2/databaseRag/artigos
Caminho do Modelo LLM: /content/drive/MyDrive/InfoBio/TCC 2/databaseRag/models/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf
Caminho do Banco de Dados Chroma: /content/drive/MyDrive/InfoBio/TCC 2/databaseRag/chroma_db


In [ ]:
# CÉLULA DE VERIFICAÇÃO DE INTEGRIDADE (CHECKSUM)
import hashlib

# Certifique-se de que a variável LLAMA_MODEL_PATH está definida com o caminho correto
# Esta variável vem da sua Célula 1.
model_file_path = LLAMA_MODEL_PATH

# Cria um objeto hash SHA256
sha256_hash = hashlib.sha256()

print(f"Calculando o checksum SHA256 para: {model_file_path}")
print("Isso pode levar um ou dois minutos para um arquivo grande...")

try:
    with open(model_file_path, "rb") as f:
        # Lê o arquivo em pedaços (chunks) para não sobrecarregar a memória RAM
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)

    calculated_checksum = sha256_hash.hexdigest()
    print("\n✅ Cálculo concluído!")
    print(f"\nChecksum Calculado: {calculated_checksum}")
    print(f"\nChecksum Oficial: 8ba9baf3a7345f705a11878397500fb25174034f0fd784e83aa4a96aaa47735f")

except FileNotFoundError:
    print(f"\nERRO: Arquivo não encontrado em '{model_file_path}'. Verifique o caminho.")
except Exception as e:
    print(f"\nOcorreu um erro: {e}")

Calculando o checksum SHA256 para: /content/drive/MyDrive/InfoBio/TCC 2/databaseRag/models/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf
Isso pode levar um ou dois minutos para um arquivo grande...

✅ Cálculo concluído!

Checksum Calculado: 8ba9baf3a7345f705a11878397500fb25174034f0fd784e83aa4a96aaa47735f

Checksum Oficial: 8ba9baf3a7345f705a11878397500fb25174034f0fd784e83aa4a96aaa47735f


In [ ]:
#CÉLULA 2 - REMOÇÃO DO ANTIGO BD

#importa biblioteca para interagir com o sistema operacional
import os
#importa a biblioteca para fazer operações de alto nível em arquivos, como apagar um diretório inteiro
import shutil


#verifica se a pasta do banco de dados ChromaDB (definida na céula 1) já existe
if os.path.exists(CHROMA_PERSIST_DIR): #caso exista
    print(f"Removendo o banco de dados antigo em: {CHROMA_PERSIST_DIR}") #informa que será excluida
    shutil.rmtree(CHROMA_PERSIST_DIR) #apaga a pasta e o conteúdo que houver dentro dela
    print("Banco de dados antigo removido com sucesso.")
else: #caso não exista, apenas informa:
    print("Nenhum banco de dados antigo encontrado para remover.")


Nenhum banco de dados antigo encontrado para remover.


In [ ]:
# CÉLULA 3 - INSTALAÇÃO DAS BIBLIOTECAS

#instalações padrão que são rápidas:
  # langchain -> framework principal
  # chromaBD -> banco de dados vetorial
  # sentence-transformers -> gerador de embeddings
  # docx2txt, pypdf, unstructured -> processamento de documentos. (unstructured é o salvador para artigos de duas colunas etc)
!pip install -q langchain langchain-community chromadb sentence-transformers docx2txt pypdf "unstructured[all-docs]"

#tentativa de instalação rápida do llama-cpp-python
print("➡️ Tentando instalar llama-cpp-python a partir de uma roda pré-compilada...")

#instala a biblioteca que permite rodar o modelo Llama (.guff), tentando encontrar uma versão pré compilada para acelerar o processo.
!pip install llama-cpp-python==0.2.77 --no-cache-dir

print("\n✅ Verificação final da instalação:")

#exibe os detalhes do pacote para confirmar que foi instalado corretamente, e verificar a versão.
!pip show llama-cpp-python

➡️ Tentando instalar llama-cpp-python a partir de uma roda pré-compilada...

✅ Verificação final da instalação:
Name: llama_cpp_python
Version: 0.2.77
Summary: Python bindings for the llama.cpp library
Home-page: https://github.com/abetlen/llama-cpp-python
Author: 
Author-email: Andrei Betlen <abetlen@gmail.com>
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: diskcache, jinja2, numpy, typing-extensions
Required-by: 


In [ ]:
# CÉLULA 4 - DOWLOAD DO MODELO
#importa biblioteca para interagir com arquivos do sistema
import os

#extrai apenas o caminho do diretório onde o arquivo do modelo deve ficar
model_dir = os.path.dirname(LLAMA_MODEL_PATH)
#cria a pasta do destino do modelo, caso ainda não exista
os.makedirs(model_dir, exist_ok=True)

#define a URL de download do modelo via Huggin Face
model_url = "https://huggingface.co/bartowski/Llama-3-8B-Instruct-GGUF/resolve/main/Llama-3-8B-Instruct.Q4_K_M.gguf"


#verifica se o modelo não existe no local esperado
if not os.path.exists(LLAMA_MODEL_PATH):
    print(f"Baixando modelo de {model_url}...")
    !wget -O "{LLAMA_MODEL_PATH}" "{model_url}" #comando para baixar o arquivo da URL e salvar no caminho definido
    print("Download concluído!")
else: #caso já exista o modelo, pula o download e puxa da pasta do drive setada na C1.
    print(f"Modelo já existe em {LLAMA_MODEL_PATH}. Pulando o download.")


Modelo já existe em /content/drive/MyDrive/InfoBio/TCC 2/databaseRag/models/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf. Pulando o download.


In [ ]:
# CÉLULA 5 - IMPORTAÇÕES


#biblioteca para interagir com arquivos e pastas do sistema
import os
import shutil
#importa divisor de texto, que quebra documentos longos em chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
#importa a classe do ChromaDB, onde os chunks serão armazenados e indexados
from langchain_community.vectorstores import Chroma
#importa um fluxo pré construido que conecta a busca de documentos com geração de respostas pelo LLM
from langchain.chains import RetrievalQA
#importa o conector que permite ao LangChain usar o modelo Llama 3 do arquivo .guff
from langchain_community.llms import LlamaCpp
#importa ferramenta que cria templates de prompt, estrtuturando a pergunta enviada ao LLM
from langchain.prompts import PromptTemplate
#importa função para limpar metadados complexos
from langchain_community.vectorstores.utils import filter_complex_metadata
#carregador de arquivos unstructured, possibilita a leitura de arquivos com diretos formatos **ponto chave p ler artigos cientificos e revistas
from langchain_community.document_loaders import UnstructuredFileLoader
#importa 'callback' para exibir a resposta do LLM em tempo real ~vai printando cfme encontra resposta~
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
#importa classe principal que carrega modelo para transformar trechos de texto em vetores numéricos (embeddings)
from langchain_community.embeddings import SentenceTransformerEmbeddings




In [ ]:
# CÉLULA 6 - FUNÇÕES DE CARREGAMENTO E CHUNKING


#define a função que aceita o caminho de uma pasta como entrada
   #Carrega documentos de uma pasta usando a biblioteca 'unstructured', que entende layouts complexos como colunas, tabelas e imagens.

def load_documents_from_folder_unstructured(folder_path):
    documents = [] #cria lista vazia para armazenamento dos elementos carregados
    for root, _, files in os.walk(folder_path): #inicia laço que percorre todas as pastas e arquivos do caminho
        for file in files: #inicia um segundo laço para processar cada arquivo encontrado individualmente
            if file.startswith("~$"): #verifica se é um arquivo temporário do Office (que deve ser ignorado)
                continue #ignora os arquivos temporários e segue laço

            file_path = os.path.join(root, file) #monta o caminho completo para o arquivo que será processado


            loader = UnstructuredFileLoader( #cria carregador de arquivos unstructured para arquivo atual
                file_path,
                mode="elements", #quebra o doc em partes lógicas (títulos, parágrafos, tabelas)
                strategy="auto"  #deixa a biblioteca escolher a melhor estratégia para extração para arquivo
            )

            try: #inicia bloco para tratamento de possíveis erros no carregamento
                print(f"Carregando com Unstructured: {file_path}") #informa qual arquivo está sendo carregado no momento
                loaded_docs = loader.load() #executa o carregamento, extraindo os elementos do arquivo

                for doc in loaded_docs: #inicia laço para verificar cada elemento extraído (parágrafo, tabelas, colunas)
                    if doc.metadata.get('category') == 'Table': #categoriza tabelas
                        doc.page_content = f"A seguir, uma tabela extraída do documento:\n{doc.page_content}"
                documents.extend(loaded_docs) #adiciona elementos extraídos do arquivo à lista principal 'documents'
            except Exception as e: #função para capturar possíveis erros
                print(f"Erro ao carregar {file_path} com Unstructured: {e}") #informa o arquivo que falhou e o respectivo erro

    print(f"\nCarregados {len(documents)} elementos dos documentos.") #informa quantos elementos foram carregados
    return documents #retorna lista completa com todos os elementos, de todos os documentos


def split_documents_into_chunks(documents, chunk_size=800, chunk_overlap=200): #define a função para 'fatiar' documentos com valores padrão de tamanho e sobreposição
    text_splitter = RecursiveCharacterTextSplitter( #cria o divisor de texto com as configurações especificadas
        chunk_size=chunk_size, #define tamanho máximo de cada pedaço de texto (em caracteres)
        chunk_overlap=chunk_overlap, #define quantos caracteres do final de um chunk se repetem no início do próximo
        length_function=len,  #informa ao divisor para medir o tamanho dos chunks usando a função padrão len()
        add_start_index=True, #adiciona aos metadados de cada chunk a sua posição de início no documento original.
    )
    chunks = text_splitter.split_documents(documents) #executa a divisão dos documentos na lista de chunks.
    print(f"Documentos divididos em {len(chunks)} chunks.") #informa o número total de chunks criados após a divisão.


    return chunks #retorna a lista final de chunks





In [ ]:
# CÉLULA 7 -

# Função 1: define a função para criar um novo banco de dados vetorial
def create_vector_db(chunks, embedding_model_name="all-MiniLM-L6-v2", persist_directory=CHROMA_PERSIST_DIR):
    embeddings = SentenceTransformerEmbeddings(model_name=embedding_model_name) #carrega modelo que transforma texto em vetores númericos (embeddings)
    filtered_chunks = filter_complex_metadata(chunks) #limpa os metadados dos chunks para evitar erros na criação do banco
    print("Criando e persistindo o ChromaDB com metadados filtrados...")
    db = Chroma.from_documents( #comando principal que cria o banco de dados a partir dos documentos
        documents=filtered_chunks, #informa quais chunks (já filtrados) devem ser adicionados ao banco
        embedding=embeddings, #define qual modelo de embedding será usado para vetorizar os chunks
        persist_directory=persist_directory #especifica a pasta onde o banco de dados final será salvo
    )
    print(f"ChromaDB criado em: {persist_directory}")
    return db #retorna o obketo do banco de dados recém-criado


#Função 2: carrega um banco de dados vetorial que já foi criado e salvo anteriormente
def load_vector_db(embedding_model_name="all-MiniLM-L6-v2", persist_directory=CHROMA_PERSIST_DIR): #define função para carregar um banco de dados que já existe
    embeddings = SentenceTransformerEmbeddings(model_name=embedding_model_name) #carrega o mesmo modelo de embedding para entender os vetores já salvos
    print(f"Carregando ChromaDB de: {persist_directory}")
    db = Chroma(persist_directory=persist_directory, embedding_function=embeddings) #cria uma instância no ChromaDB carregando os dados da pasta específica
    return db #retorna o objeto do banco de dados carregado, pronto para uso



#Função 3: monta e conecta todos os componnetes: o LLM, o retriever e o prompt, criando o RAG final
def setup_rag_chain(vector_db, llm_model_path): #define função que configura o fluxo de pergunta e resposta
    llm = LlamaCpp( #carrega modelo LLama 3 com as configurações:
        model_path=llm_model_path,
        temperature=0.2, #mais focado, menos criativo para respostas + precisas
        max_tokens=1024, #define o tamanho máximo da resposta gerada pelo modelo
        n_ctx=8192, #define tamanho da 'memória' contexto que o modelo pode usar
        n_gpu_layers=-1, #tenta usar GPU para processar o máximo de camadas do modelo, acelerando a resposta
        verbose=False, #desativa logs internos do Llama para manter saída limpa
        streaming=True, #ativa resposta em tempo real, palavra por palavra
        callbacks=[StreamingStdOutCallbackHandler()] #conexação da função para exibir resposta em tempo real
    )


    retriever = vector_db.as_retriever( #cria o recuperador de documentos a partir do banco de dados
        search_type="mmr", #usa método MMR para buscar documentos relevantes entre si
        search_kwargs={"k": 5, "fetch_k": 20} #configura MMR para buscar 20 chunks e selecionar os 5 melhores
    )

    #define o modelo de prompt instruindo como o LLM deve se comportar
    prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Você é um profissional de saúde, especialista em orientação pré e pós-operatória de cirurgia cardíaca.
Forneça respostas padronizadas, seguras, objetivas e em linguagem acessível ao paciente. **Responda de forma direta, omitindo saudações ou introduções robóticas.** Suas orientações são estritamente baseadas no contexto fornecido.
Se a informação for individualizada (como dosagem, diagnóstico ou liberação para atividades) ou não estiver no contexto, responda com uma das seguintes frases padronizadas:
'Esta é uma questão específica do seu histórico médico e deve ser discutida diretamente com seu médico cardiologista ou a equipe de enfermagem.' OU 'Essa informação depende de uma avaliação individualizada.
Por favor, consulte seu médico ou a equipe de saúde responsável.'
<|eot_id|><|start_header_id|>user<|end_header_id|>
Com base no contexto abaixo, responda à minha pergunta.

Contexto:
{context}

Pergunta:
{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
    #transforma a string do prompt em um objeto que o LangChain pode usar
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    qa_chain = RetrievalQA.from_chain_type( #cria cadeia RAG final, unindo todas as peças
        llm=llm, #conecta LLM a cadeia
        chain_type="stuff", #define que todos os textos recuperados serão compilados no prompt de uma vez
        retriever=retriever, #conecta o recuperador MMR a cadeia
        return_source_documents=True, #faz com que a cadeira retorne também os trechos dos documentos usads na resposta
        chain_type_kwargs={"prompt": PROMPT} #insere o prompt customizado na cadeia
    )
    print("Cadeia RAG configurada com sucesso (MMR e Streaming ativados).")
    return qa_chain #retorna a cadeia RAG completa e pronta para receber perguntas

In [ ]:
# CÉLULA 8 (VERSÃO FINAL CORRIGIDA) - PREPARAÇÃO E CARREGAMENTO DO BANCO DE DADOS

import os
import shutil
import tempfile
from langchain_community.vectorstores import Chroma # Necessário se não estiver importado em Célula anterior
from langchain_community.embeddings import SentenceTransformerEmbeddings # Necessário se não estiver importado
from langchain_community.docstore.document import Document # Necessário se não estiver importado

# Supondo que essas funções e variáveis (LLAMA_MODEL_PATH, CHROMA_PERSIST_DIR, DATA_PATH)
# estão definidas em células anteriores ou são variáveis globais.

# Variável principal para armazenar o objeto do banco de dados
vector_db = None

# Define a pasta temporária local para criar o ChromaDB
temp_chroma_dir = os.path.join(tempfile.gettempdir(), "chroma_temp_db")

# --- FUNÇÕES (Requisito: Definições de load_documents_from_folder_unstructured, split_documents_into_chunks, create_vector_db, load_vector_db devem estar em células anteriores) ---

# PARTE 1: Lógica de carregamento/criação do DB
# Prioriza carregar do local final (Drive) se ele existir

if os.path.exists(CHROMA_PERSIST_DIR):
    print("ChromaDB existente detectado no Google Drive. Tentando carregar...")
    try:
        # Tenta carregar o DB do Drive
        vector_db = load_vector_db(persist_directory=CHROMA_PERSIST_DIR)
        if vector_db:
            print("ChromaDB carregado com sucesso do Google Drive.")

    except Exception as e:
        print(f"❌ Erro ao carregar ChromaDB do Google Drive: {e}")
        print("A causa pode ser permissão ou corrupção. Tentando recriar o banco de dados...")
        vector_db = None # Reset para forçar a criação


if not vector_db: # Se o carregamento falhou ou o DB não existia
    print("ChromaDB não encontrado ou falha ao carregar. Iniciando processo de criação em diretório temporário...")

    # Garante que o diretório temporário esteja limpo para evitar erros de escrita (Código 14)
    if os.path.exists(temp_chroma_dir):
        print(f"Removendo diretório temporário antigo: {temp_chroma_dir}")
        shutil.rmtree(temp_chroma_dir)

    # Requer as funções de carregamento e split de células anteriores
    documents = load_documents_from_folder_unstructured(DATA_PATH)

    if not documents:
        print(f"❌ ERRO CRÍTICO: Nenhum documento processado em {DATA_PATH}. Verifique a pasta.")
    else:
        chunks = split_documents_into_chunks(documents)
        print(f"Documentos divididos em {len(chunks)} chunks.")

        # Cria DB no diretório temporário
        vector_db = create_vector_db(chunks, persist_directory=temp_chroma_dir)

        if vector_db:
            print(f"ChromaDB criado com sucesso no diretório temporário: {temp_chroma_dir}")

            # --- PONTO CRÍTICO DE CORREÇÃO ---
            # 1. Fechar a conexão do DB antes de copiar, liberando locks do SO
            del vector_db
            vector_db = None
            print("Conexão com o banco de dados temporário fechada.")

            # 2. Lógica de cópia para o Google Drive
            print(f"Copiando ChromaDB do diretório temporário para o Google Drive: {CHROMA_PERSIST_DIR}")

            if os.path.exists(CHROMA_PERSIST_DIR):
                 print(f"Removendo conteúdo existente em {CHROMA_PERSIST_DIR} antes de copiar.")
                 shutil.rmtree(CHROMA_PERSIST_DIR)
            try:
                # Copia o DB para o Drive
                shutil.copytree(temp_chroma_dir, CHROMA_PERSIST_DIR)
                print("Cópia para o Google Drive concluída com sucesso.")

                # 3. Recarregar o DB diretamente do Drive para uso subsequente
                vector_db = load_vector_db(persist_directory=CHROMA_PERSIST_DIR)

            except Exception as e:
                 print(f"❌ ERRO ao copiar ou recarregar do Google Drive: {e}")
                 vector_db = None # Garante que o estado final seja de erro se o DB não persistir
        else:
            print("\n❌ Falha na criação do ChromaDB no diretório temporário.")


# PARTE 2: Verificação final
if vector_db:
    print("\n✅ Ambiente pronto! A variável 'vector_db' foi carregada/criada com sucesso.")
    print("   Prossiga para a Célula 9 (Console) ou Célula 10 (Gradio).")
else:
    print("\n❌ ERRO CRÍTICO: Falha ao preparar o banco de dados vetorial. O chatbot não pode iniciar.")

ChromaDB não encontrado ou falha ao carregar. Iniciando processo de criação em diretório temporário...
Carregando com Unstructured: /content/drive/MyDrive/InfoBio/TCC 2/databaseRag/artigos/5371-Texto do artigo-16016-1-10-20081007.pdf


Carregando com Unstructured: /content/drive/MyDrive/InfoBio/TCC 2/databaseRag/artigos/4284-28572-1-PB.pdf
Carregando com Unstructured: /content/drive/MyDrive/InfoBio/TCC 2/databaseRag/artigos/IIDD225 PDF.pdf
Carregando com Unstructured: /content/drive/MyDrive/InfoBio/TCC 2/databaseRag/artigos/rgenfermagem,+551-556.pdf
Carregando com Unstructured: /content/drive/MyDrive/InfoBio/TCC 2/databaseRag/artigos/35284-Texto do Artigo-253541-1-10-20241114.pdf
Carregando com Unstructured: /content/drive/MyDrive/InfoBio/TCC 2/databaseRag/artigos/ijrb-17-883.pdf
Carregando com Unstructured: /content/drive/MyDrive/InfoBio/TCC 2/databaseRag/artigos/ali,+2+CUIDADOS+DE+ENFERMAGEM+A+PACIENTES+EM+PRÉ-OPERATÓRIO+PROPOSTA+DE+CHECKLIST.pdf
Carregando com Unstructured: /content/drive/MyDrive/InfoBio/TCC 2/databaseRag/artigos/afab146.pdf
Carregando com Unstructured: /content/drive/MyDrive/InfoBio/TCC 2/databaseRag/artigos/submetido-ARTIGO+CIRURGIA+CARDÍACA (1).pdf
Carregando com Unstructured: /content/drive

InternalError: Database error: error returned from database: (code: 14) unable to open database file

In [ ]:
# CÉLULA 10 (VERSÃO FINAL CORRIGIDA) - INTERFACE DE CHAT (VERSÃO WEB COM GRADIO)

# Garante que o Gradio está instalado
!pip install -q gradio

import gradio as gr
import os
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# --- Função para configurar a cadeia RAG para a interface ---
# Recebe o objeto do prompt pronto para evitar erros de validação.
def setup_rag_chain_for_gradio(vector_db, llm_model_path, prompt_object):
    llm_gradio = LlamaCpp(
        model_path=llm_model_path,
        temperature=0.2, # mais focado, menos criativo para respostas + precisas
        max_tokens=1024,
        n_ctx=8192,
        n_gpu_layers=-1, # tenta usar a GPU para máxima aceleração
        verbose=False,
        streaming=True   # Mantém o streaming para gerar a resposta em tempo real
    )

    retriever = vector_db.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 5, "fetch_k": 20}
    )

    # A cadeia agora usa o objeto do prompt que foi passado diretamente
    qa_chain_gradio = RetrievalQA.from_chain_type(
        llm=llm_gradio,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt_object}
    )
    return qa_chain_gradio

# --- Função que o Gradio vai chamar para cada mensagem do usuário ---
def chat_function(message, history):
    print(f"Pergunta recebida pela interface: {message}")

    # O método .stream retorna um "gerador" que produz os tokens da resposta um por um
    response_generator = qa_chain_for_ui.stream({"query": message})

    full_response = ""
    # Iteramos sobre o gerador para construir a resposta em tempo real na interface
    for chunk in response_generator:
        # A resposta do LLM vem na chave 'result'
        if 'result' in chunk:
            token = chunk['result']
            full_response += token
            yield full_response # 'yield' envia o texto atualizado para a UI do Gradio

# --- Lógica principal para lançar a interface ---
# Verifica se o banco de dados (da Célula 8) e o modelo LLM estão prontos
if 'vector_db' in locals() and vector_db and os.path.exists(LLAMA_MODEL_PATH):
    print("Configurando a cadeia RAG para a interface Gradio...")

    # 1. Definimos a string completa do template do prompt (CORRIGIDO: Removido indentação extra)
    prompt_template_string = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Você é um profissional de saúde, especialista em orientação pré e pós-operatória de cirurgia cardíaca. Forneça respostas padronizadas, seguras, objetivas e em linguagem acessível ao paciente. **Nunca mencione que está buscando em documentos, base de conhecimento ou contexto.** Mantenha um tom humano e profissional. Se a informação for individualizada (como dosagem, diagnóstico ou liberação para atividades) ou não estiver no contexto, responda com uma das seguintes frases padronizadas: 'Esta é uma questão específica do seu histórico médico e deve ser discutida diretamente com seu médico cardiologista ou a equipe de enfermagem.' OU 'Essa informação depende de uma avaliação individualizada. Por favor, consulte seu médico ou a equipe de saúde responsável.'
<|eot_id|><|start_header_id|>user<|end_header_id|>
Com base no contexto abaixo, responda à minha pergunta.

Contexto:
{context}

Pergunta:
{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

    # 2. Criamos o objeto PromptTemplate fora da função (CORRIGIDO: Corrigido o nome da variável de prompt)
    PROMPT_FOR_GRADIO = PromptTemplate(
        template=prompt_template_string,
        input_variables=["context", "question"]
    )

    # 3. Passamos o objeto já pronto para a função de configuração
    qa_chain_for_ui = setup_rag_chain_for_gradio(vector_db, LLAMA_MODEL_PATH, PROMPT_FOR_GRADIO)

    print("Cadeia RAG pronta.")

    print("\n🚀 Lançando a interface do Gradio...")
    print("Aguarde os links aparecerem. Clique no link PÚBLICO (Public URL) para abrir e compartilhar.")

    # 4. Chamada do ChatInterface
    chat_interface = gr.ChatInterface(
        fn=chat_function,
        title="🤖 Chatbot de Orientação Cardíaca",
        description="Faça uma pergunta sobre cuidados pré ou pós-operatórios cardíacos.",
        examples=[["Quais os cuidados com a ferida operatória?"], ["Posso dirigir após a cirurgia?"], ["Como devo me alimentar antes da cirurgia?"]],
        theme="soft"
    )

    # Lança a interface, gerando o link público para compartilhamento
    chat_interface.launch(share=True, debug=True)

else:
    print("❌ ERRO: Execute a Célula 8 primeiro para preparar o ambiente (vector_db).")

In [ ]:
# CÉLULA 9- INTERFACE DE CHAT (VERSÃO CONSOLE - OPCIONAL)

if vector_db and os.path.exists(LLAMA_MODEL_PATH):
    # Configura a cadeia RAG para o console (com o callback de streaming original)
    qa_chain_console = setup_rag_chain(vector_db, LLAMA_MODEL_PATH)

    print("\n======================================================================")
    print("Chatbot de Orientação Cardíaca (digite 'sair' para encerrar)")
    print("======================================================================")

    while True:
        user_query = input("\nSua pergunta: ")
        if user_query.lower() == 'sair':
            print("Encerrando o chatbot. Até logo!")
            break

        print("\nResposta: ", end="", flush=True)
        # O streaming é tratado pelo callback dentro de setup_rag_chain
        result = qa_chain_console.invoke({"query": user_query})

        # Impressão das fontes
        print("\n\n---")
        print("Fontes consultadas:")
        sources = {os.path.basename(doc.metadata.get('source', 'N/A')) for doc in result['source_documents']}
        for source in sorted(list(sources)):
            print(f"- {source}")
        print("---")
else:
    print("❌ Execute a Célula 8 primeiro para preparar o ambiente.")